<a href="https://colab.research.google.com/github/jazkre/NAEX/blob/main/code/01NAEX_Exercise_05_R_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# O1NAEX Exercise 05

J.Kreanová

In [ ]:
cat(system('sudo apt-get install -y gmp', intern=TRUE), sep = "\n")
cat(system('sudo apt-get install -y partitions', intern=TRUE), sep = "\n")
cat(system('sudo apt-get install -y DoE.base', intern=TRUE), sep = "\n")

list_of_packages <- c("tidyverse","MASS","FrF2","pid","rsm") # "agricolae","nortest","lattice","pwr",
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:
tibble::tibble(
  Package = names(installed.packages()[,3]),
  Version = unname(installed.packages()[,3])
)

##	Problem 6.1
from the chapter 6, D. C. Montgomery DAoE - 8. edition.

An engineer is interested in the effects of cutting
speed (A), tool geometry (B), and cutting angle (C) on the life
(in hours) of a machine tool. Two levels of each factor are
chosen, and three replicates of a 23 factorial design are run.
The results are as follows:

In [ ]:
df61 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Problem_6_1.txt",header=TRUE,sep=";")
head(df61)


1. Estimate the factor effects. Which effects appear to be
large?
2. Use the analysis of variance to confirm your conclusions
for part (a).
3. Write down a regression model for predicting tool life
(in hours) based on the results of this experiment.
4. Analyze the residuals. Are there any obvious problems?
5. On the basis of an analysis of main effect and interaction
plots, what coded factor levels of A, B, and C
would you recommend using?
6. Reconsider task 3. Use the regression
model to generate response surface and contour plots of the
tool life response. Interpret these plots. Do they provide insight
regarding the desirable operating conditions for this process?
7. Find the standard error of the factor effects and
approximate 95 percent confidence limits for the factor effects. Do the results of this analysis agree with the conclusions from the analysis of variance?



# 0 .Data Preprocessing

In [ ]:
df61

In [ ]:
str(df61)

In [ ]:
df61$A=factor(df61$CuttingSpeed)
df61$B=factor(df61$ToolGeometry)
df61$C=factor(df61$CuttingAngle)

### $2^3$ factorial design  

**A** - Cutting Speed

**B** - Tool Geometry

**C** - Cutting Angle



In [ ]:
df61=df61[, c('A','B','C','LifeHours')]
head(df61)

In [ ]:
tb<- df61 %>%
  group_by(A, B, C) %>%
  summarise(RI = LifeHours[1], RII = ifelse(n() >= 2, LifeHours[2], NA), RIII = ifelse(n() == 3, LifeHours[3], NA)) %>%
  ungroup()

In [ ]:
tb$Total=tb$RI+tb$RII+tb$RIII
tb$notation<-c('(1)','c','b','bc','a','ac','ab','abc')
tb

In [ ]:
df <- as.data.frame(tb)
df
#str(df)

# 1. Factor Effects
Estimate the factor effects. Which effects appear to be large?

main effect of A: $A = \frac{1}{4n} (a + ab + ac + abc − (1) − b − c − bc)$

main effect of B: $B =  \frac{1}{4n} (b + ab + bc + abc − (1) − a − c − ac)$

main effect of C: $C = \frac{1}{4n} (c + ac + bc + abc − (1) − a − b − ab)$

interaction of AB: $AB =  \frac{1}{4n}(abc + ab + c + (1) − a − b − ac − bc)$

interaction of AC: $AC =  \frac{1}{4n} (abc + ac + b + (1) − a − c − ab − bc)$

interaction of BC: $BC =  \frac{1}{4n} (abc + bc + a + (1) − b − b − ab − ac)$

interaction of ABC: $ABC =  \frac{1}{4n}(abc + a + b + c − ab − ac − bc − (1))$

In [ ]:
n=3 #replikacie
a <- df$Total[df$notation == 'a']
b <- df$Total[df$notation == 'b']
c <- df$Total[df$notation == 'c']
ab <- df$Total[df$notation == 'ab']
bc <- df$Total[df$notation == 'bc']
ac <- df$Total[df$notation == 'ac']
abc <- df$Total[df$notation == 'abc']
e <- df$Total[df$notation == '(1)']

In [ ]:
df$effect[df$notation=='a']=(a + ab + ac + abc - e - b - c - bc)/(4*n)
df$effect[df$notation=='b']=(b + ab +bc + abc -e -a -c -ac)/(4*n)
df$effect[df$notation=='c']=(c + ac +bc +abc -e -a -b -ab)/(4*n)
df$effect[df$notation=='ab']=(abc + ab +c +e -a -b-ac-bc)/(4*n)
df$effect[df$notation=='ac']=(abc +ac + b +e -a -c -ab -bc)/(4*n)
df$effect[df$notation=='bc']=(abc +bc +a +e -b -c -ab -ac)/(4*n)
df$effect[df$notation=='abc']=(abc +a +b +c -ab -ac -bc -e)/(4*n)

In [ ]:
df

Large appearing effects are for Tool geometry B (11.33) and for a Speed-Angle iteration AC (-8.83).

On contrary Cutting speed A effect appears to be small in comparison to other (0.33).

# 2. ANOVA
Use the analysis of variance to confirm your conclusions for part (a).

In [ ]:
summary(aov(LifeHours ~ A*B*C, df61))

From Anova, we can confirm significance of B,C and AC interaction on a significance level 0.01.

We can reduce the model to A,B,C,AC. A is included to keep hierarchy.

In [ ]:
aov(LifeHours ~ A+B+C +A:C, df61)

In [ ]:
summary(aov(LifeHours ~ A+B+C +A:C, df61))

Significance: B,C,AC at 0.01 (1%)

# 3. Regression model
Write down a regression model for predicting tool life (in hours) based on the results of this experiment.

-model without distinguishing replication order

In [ ]:
m1=lm(LifeHours ~ A+B+C +A:C, df61)
summary(m1)

#summary(lm(lh~A*C+B,rs))

$y_{ijk} =27.167 + 9.167 x_{A} +11.333 x_{b} + 15.667 x_{c} -17.667x_{A}x_{c}$

In [ ]:
par(mfrow = c(2, 2))
plot(lm(LifeHours ~ A+B+C +A:C, df61))
par(mfrow = c(1, 1))

-- model that takes into account a form of data measurement by replications:

We wil use [FrF2() ](https://www.r-bloggers.com/2011/05/fractional-factorial-designs-using-frf2/) function to generate apropriate class

In [ ]:
#extract Life Hours
lh<-c(tb$RI,tb$RII,tb$RIII)
lh

rs=rbind(df,df,df)
rs=rs[,c('A','B','C')]
rs <- rs %>%
  mutate(Block = case_when(
    row_number() <= 8 ~ '.1',
    row_number() <= 16 ~ '.2',
    TRUE ~ '.3'
  ))
rs$lh=lh
#rs

In [ ]:
mFrF2 <-  FrF2(2^n, n, replications = 3, randomize = FALSE,factor.names = c("A", "B", "C"))
#we need to set design matrix thatcorresponds with our response: life hours
mFrF2$A<-rs$A
mFrF2$B<-rs$B
mFrF2$C<-rs$C
mFrF2 <- add.response(mFrF2,response=lh)
mFrF2
MEPlot(mFrF2)
IAPlot(mFrF2)

In [ ]:
summary(lm(lh~A+B+C+A:C,mFrF2))

$y_{ijk} =40.83 + 3.42 x_{A} +5.67 x_{b} + 0.17 x_{c} -4.2x_{A}x_{c}$

# 4. Residuals
Analyze the residuals. Are there any obvious problems?

In [ ]:
par(mfrow = c(2, 2))
plot(lm(lh~A+ B +C +A:C,mFrF2))
par(mfrow = c(1, 1))

Residuals do not depict any obvious problems

In [ ]:
qqplot(DanielPlot(mFrF2,alpha=0.1)$x,DanielPlot(mFrF2)$y)


# 5. Interaction plots
On the basis of an analysis of main effect and interaction plots, what coded factor levels of A, B, and C would you recommend using?

Due to decreasing effect of interaction AC, the AC plot would be most revealing. The plot shows that in high level of C and low level A the life hours are maximized. (Low level of Cutting Speed and High level of Cutting angle)

B increases lihe hours due to its positive effect.

In [ ]:
colors <- c("red", "black")

interaction.plot(mFrF2$A,mFrF2$C,lh,type = "l", col = colors, pch = 19,
                 xlab = "C Cutting Angle", ylab = "life hours", main = "Interaction Plot")
legend("topright", legend = levels(mFrF2$C), fill = colors,title="A factor")
interaction.plot(mFrF2$C,mFrF2$A,lh, col = colors, pch = 19,
                 xlab = "A Cutting Speed", ylab = "life hours", main = "Interaction Plot")
legend("topright", legend = levels(mFrF2$A), fill = colors, title="C factor")

interaction.plot(mFrF2$B,mFrF2$C,lh, col = colors, pch = 19,
                 xlab = "B Tool Geometry", ylab = "life hours", main = "Interaction Plot")
legend("topright", legend = levels(mFrF2$C), fill = colors, title="C factor")

attach(mFrF2)

# 6. Response Surface, Contour plot
Reconsider task 3. Use the regression model to generate response surface and contour plots of the tool life response. Interpret these plots. Do they provide insight regarding the desirable operating conditions for this process?

In [ ]:
#to construct these plots in terms of the natural factor levels
A.num <- as.numeric(as.character(A))
B.num <- as.numeric(as.character(B))
C.num <- as.numeric(as.character(C))

df61.num <- as.data.frame(cbind(lh,A.num,B.num,C.num))
str(df61.num)

In [ ]:
m_num<- lm(lh~A.num*C.num+B.num)
summary(m_num)

$y_{ijk} =40.83 + 3.42 x_{A} +5.67 x_{b} + 0.17 x_{c} -4.2x_{A}x_{c}$

Model has not changed due to fact, that original level values count +1 or -1 naturally. We would see a change with different values of levels.

In [ ]:
#contourPlot(lsmodel,
#            xlab=attr(lsmodel$terms,'term.labels')[1],
#            ylab=attr(lsmodel$terms,'term.labels')[2],
#            main="Contour plot",
#            N=25,
#            xlim=c(-3.2, 3.2),
#            ylim=c(-3.2, 3.2),
#            colour.function=terrain.colors)

In [ ]:
contourPlot(m_num,
            xlab="A.num",ylab="C.num",
            main="Contour plot of LifeHours",
            xlim=c(-1.1, 1.1),
            ylim=c(-1.1, 1.1),
            colour.function=terrain.colors)

In [ ]:
rsm_abc<-rsm(lh~FO(A.num,C.num)+TWI(A.num,C.num), data=df61.num)

In [ ]:
persp(rsm_abc, A.num~C.num, zlab = "lh", main="Response surface?")

Plots show dependance of Life Hours to A and C. These plots allow us to show the lifehours based on AC interaction. We can observe high point of LH.

# 7. Standard error of factor effects
 Find the standard error of the factor effects and approximate 95 percent confidence limits for the factor effects. Do the results of this analysis agree with the conclusions from the analysis of variance?

Standard error of model coefficients:

$se( \hat{β}) =\sqrt{\frac{\sigma^{2}}{2n^{k}}}=\sqrt{\frac{MS_{E}}{2n^{k}}}=1.38$

The standard errors of all model coefficients are equal because
the design is orthogonal.

In [ ]:
summary(aov(lh~A+B+C+A:C))

In [ ]:
MSe=summary(aov(lh~A+B+C+A:C))[[1]]$"Mean Sq"[5]
se=sqrt(MSe/(2*2^3))
se

In [ ]:
summary(lm(lh~A+B+C+A:C))

Confidence interval on model coefficients:

$\hat{β} − t_{α/2,df_{E}} se( \hat{β}) ≤ β ≤ \hat{β} + t_{α/2,df_{E}} se(\hat{β})$



In [ ]:
dfe=length(lh)-5
shift=MSe* qt(0.975, df = dfe)

#lm(lh~A+B+C+A:C)$coefficients

lh.lm<-lm(lh~A+B+C+A:C)
lh.CIl<-lh.lm$coefficients-shift
lh.CIu<-lh.lm$coefficients+shift

In [ ]:
#tibble(lh.lm$coefficients,lh.CIl,lh.CIu)


The estimated standard error would be found by replacing σ^2  by its estimate S^2 and taking the
square root of this last expression:

$se(Effect) = \frac{2S}{n2^{k}}, S^{2}\doteq MS_{E}$

The 100(1 - α	) percent confidence intervals on the effects are:

$Effect \pm t_{α/2,df_{E}} se( \hat{β}) $

In [ ]:
head(df)

In [ ]:
es=2*sqrt(MSe)/sqrt(2*2^3)
df$CI_l<-df$effect-es* qt(0.975, df = dfe)
df$CI_u<-df$effect+es* qt(0.975, df = dfe)

In [ ]:
df <-df %>%
  mutate(signif_ef = ifelse(
    (CI_l*CI_u)>0, '*', ''
  ))
  df

As our previous analysis, confidence intervals for **Effects** of B,C and AC do not contain zero which means, the mentioned effects significant.